In [1]:
!pip install transformers

In [2]:
import torch
from torch.utils.data import Dataset,DataLoader
import numpy as np
import os
import copy
import cv2
import json
import transformers
from transformers import AutoImageProcessor, ViTModel
from PIL import Image, ImageMath
from transformers import AutoModel
import torch.nn as nn
from transformers import AutoTokenizer, BertModel
from torch.optim import Adam
from tqdm import tqdm
import torch.nn.functional as F
import pandas as pd
from ast import literal_eval
from torchvision.models import resnet50,SwinTransformer
import torchvision
from transformers import AutoFeatureExtractor, ResNetForImageClassification
from transformers import AutoImageProcessor, AutoModelForImageClassification
import requests
from transformers import RobertaTokenizer, RobertaModel

In [3]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device

'cuda:0'

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
def save_checkpoint(checkpoint_directory, epoch, model, optimizer, LOSS, checkpoint_name=None):
    """
    The checkpoint will be saved in `checkpoint_directory` with name `checkpoint_name`.
    If `checkpoint_name` is None, the checkpoint will be saved with name `next_checkpoint_name_id + epoch`.
    """
    if checkpoint_directory is not None:
        if checkpoint_name is None:
            checkpoint_name = f'{epoch}{CHECKPOINT_EXTENSION}'

        path = os.path.join(checkpoint_directory, checkpoint_name)
        print("path: "+str(path))
        torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'loss': LOSS,
                    }, path)
def load_checkpoint(PATH, model, optimizer):
    checkpoint = torch.load(PATH)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']

    return epoch, model, optimizer, loss
def read_json(parient_dir, name=None):
    # Opening JSON file
    if name == None:
        path = parient_dir
    else:
        path = os.path.join(parient_dir, str(name) + ".json")
    with open(path, 'r') as openfile:

        # Reading from json file
        json_object = json.load(openfile)
    return json_object
# img_dir = '/content/drive/MyDrive/HaruLab/Dataset/Musti/test/img'
# json_file =
# values = json_file['pairs']

def load_image_from_url(url):
    try:
        response = requests.get(url)
        response.raise_for_status()

        # Open the image using Pillow
        image = Image.open(BytesIO(response.content))
        return image
    except requests.exceptions.RequestException as e:
        print(f"Error loading image from URL: {e}")
        return None

## . Dataset&DataLoader

In [6]:
img_dir = '/content/drive/MyDrive/HaruLab/Dataset/Musti/train/img'
csv_file_path = '/content/drive/MyDrive/HaruLab/Dataset/Musti/train/aug_train_balan.json'
checkpoint_directory = "/content/drive/MyDrive/MMM CONFERENCE/checkpoint/CO_RES_VIT_BERT"
CHECKPOINT_EXTENSION = '.pt'

In [7]:
image_processor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k")
# tokenizer = AutoModel.from_pretrained("google/mt5-base")
tokenizer = RobertaTokenizer.from_pretrained('roberta-large')

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

In [8]:
class Dataset_MMM(Dataset):
    def __init__(self, file_path,image_dir ,image_processor,tokenizer):
        super(Dataset_MMM, self).__init__()
        self.image_dir = image_dir
        data = read_json(file_path)
        self.data = []
        for obj in data:
          if os.path.exists(os.path.join(self.image_dir, obj['image'])):
            self.data.append(obj)
        self.image_processor = image_processor
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        img_path = os.path.join(self.image_dir,self.data[index]['image'])
        text = self.data[index]['text']
        target = self.data[index]['subtask1_label']
        if target == 'NO':
          target = 0
        else:
          target = 1
        # process_image
        img = Image.open(img_path)
        if img.mode == 'L':
          img = img.convert('RGB')
        img = self.image_processor(images = img, return_tensors="pt",padding=True)
        # process_text
        text = self.tokenizer(text,padding="max_length", max_length=197,truncation=True, return_tensors="pt")

        return img, text, target

In [9]:
dataset_train = Dataset_MMM(csv_file_path,img_dir,image_processor,tokenizer)
dataloader_train = DataLoader(dataset_train,batch_size=16,drop_last=True)

## Model

### Image

In [10]:
vit_encoder = ViTModel.from_pretrained("google/vit-base-patch16-224-in21k")
resnet_encoder = ResNetForImageClassification.from_pretrained("microsoft/resnet-34")
resnet_encoder = nn.Sequential(*list(resnet_encoder.children())[:-1])

pytorch_model.bin:   0%|          | 0.00/346M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/69.5k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/87.3M [00:00<?, ?B/s]

In [11]:
swintransformer = AutoModelForImageClassification.from_pretrained("microsoft/swin-tiny-patch4-window7-224")
swintransformer = nn.Sequential(*list(swintransformer.children())[:-1])
# swintransformer

config.json:   0%|          | 0.00/71.8k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/113M [00:00<?, ?B/s]

In [12]:
# out = vit_encoder(dataset_train[0][0]['pixel_values'])
# print(out.last_hidden_state.shape)

In [13]:
# out_1 = resnet_encoder(dataset_train[0][0]['pixel_values'])
# print(out_1.last_hidden_state.shape)

In [14]:
# out_2 = swintransformer(dataset_train[0][0]['pixel_values'])
# print(out_2.last_hidden_state.shape)

In [15]:
class EarlyStopper:
    def __init__(self, patience=1, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = float('inf')

    def early_stop(self, validation_loss):
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.counter = 0
        elif validation_loss > (self.min_validation_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False
early_stopper = EarlyStopper(patience=10, min_delta=0)

### Text

In [16]:
# text_encoder = BertModel.from_pretrained("google/mt5-base").to(device)
text_encoder = RobertaModel.from_pretrained('roberta-large').to(device)

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
encoder_layer = text_encoder.encoder
text_embedding= text_encoder.embeddings
pooling_layer= text_encoder.pooler

### CrossModel

In [18]:
class CrossModel(nn.Module):
  def __init__(self,image_encoder,text_encoder,text_embedding, pooling_layer):
    super(CrossModel,self).__init__()
    #image_encoder
    self.vit_encoder = image_encoder[0]
    self.resnet_encoder = image_encoder[1]
    self.swintransformer_encoder = image_encoder[2]

    #text_encoder & text embedding
    self.text_encoder = text_encoder
    self.text_embedding = text_embedding
    self.pooling_layer = pooling_layer

    # layernorm
    self.layernorm_1_0 = nn.LayerNorm(49)
    self.layernorm_1_1 = nn.LayerNorm(512)
    self.layernorm_2_0 = nn.LayerNorm(197)

    self.layernorm_3_0 = nn.LayerNorm(1024)
    #linear
    self.linear = nn.Linear(2,1)

    # projection
    self.resnet_projection_1 = nn.Linear(49,1024)
    self.resnet_projection_2 = nn.Linear(512,197)

    self.swintransformer_projection = nn.Linear(49,197)
    self.final_projection = nn.Linear(1024,1)

  def forward(self,img,text):
    batch_size = img.size(0)
    #image_features
    image_features_0 = self.vit_encoder(img).last_hidden_state #torch.Size([16, 197, 768])
    image_features_1 = self.resnet_encoder(img).last_hidden_state #torch.Size([16, 512, 7, 7])
    image_features_2 = self.swintransformer_encoder(img).last_hidden_state #torch.Size([16, 49, 768])

    print("image_features_0"+str(image_features_0.shape))

    print("image_features_1"+str(image_features_1.shape))

    print("image_features_2"+str(image_features_2.shape))

    #projection_1
    image_features_1 = image_features_1.reshape(batch_size,512,-1) #torch.Size([16, 512, 49])
    image_features_1 = self.resnet_projection_1(image_features_1) #torch.Size([16, 512, 1024])
    image_features_1 = self.layernorm_1_0(image_features_1)
    image_features_1 = image_features_1.permute(0,2,1) #torch.Size([16, 1024, 512])
    image_features_1 = self.resnet_projection_2(image_features_1) #torch.Size([16, 1024, 197])
    image_features_1 = self.layernorm_1_1(image_features_1)
    image_features_1 = image_features_1.permute(0,2,1) #torch.Size([16, 197, 1024])
    print("image_features_1"+str(image_features_1.shape))

    #projection_2
    image_features_2 = image_features_2.permute(0,2,1) #torch.Size([16, 768, 49])
    image_features_2 = self.swintransformer_projection(image_features_2) #torch.Size([16, 768, 197])
    image_features_2 = self.layernorm_2_0(image_features_2)
    image_features_2 = image_features_2.permute(0,2,1) #torch.Size([16, 197, 768])
    print("image_features_2"+str(image_features_2.shape))

    #text_features
    text_embedding = self.text_embedding(text)
    print("text_embedding"+str(text_embedding.shape))
    final_features_0 = self.layernorm_3_0(self.linear(torch.cat(image_features_0.unsqueeze(-1) , text_embedding.unsqueeze(-1))).squeeze(-1) + text_embedding)
    final_features_1 = self.layernorm_3_0(self.linear(torch.cat(image_features_1.unsqueeze(-1) , text_embedding.unsqueeze(-1))).squeeze(-1) + text_embedding)
    final_features_2 = self.layernorm_3_0(self.linear(torch.cat(image_features_2.unsqueeze(-1) , text_embedding.unsqueeze(-1))).squeeze(-1) + text_embedding)
    print("final_features_0"+str(final_features_0.shape))
    print("final_features_1"+str(final_features_1.shape))
    print("final_features_2"+str(final_features_2.shape))


    final_features_0 = self.pooling_layer(self.text_encoder(final_features_0).last_hidden_state)
    final_features_1 = self.pooling_layer(self.text_encoder(final_features_1).last_hidden_state)
    final_features_2 = self.pooling_layer(self.text_encoder(final_features_2).last_hidden_state)
    print("final_features_0"+str(final_features_0.shape))
    print("final_features_1"+str(final_features_1.shape))
    print("final_features_2"+str(final_features_2.shape))


    image_features_1 = self.final_projection(final_features_0.permute(0,2,1)).squeeze(-1)
    image_features_2 = self.final_projection(final_features_1.permute(0,2,1)).squeeze(-1)
    image_features_3 = self.final_projection(final_features_2.permute(0,2,1)).squeeze(-1)
    print("image_features_1"+str(image_features_1.shape))
    print("image_features_2"+str(image_features_2.shape))
    print("image_features_3"+str(image_features_3.shape))

    logits_0 = F.cosine_similarity(final_features_0,image_features_1, dim =-1)
    logits_1 = F.cosine_similarity(final_features_1,image_features_2, dim =-1)
    logits_2 = F.cosine_similarity(final_features_2,image_features_3, dim =-1)
    logits = (logits_0 + logits_1 + logits_1)/3
    return logits

## Train

In [19]:
def train(train_loader, model,device, epochs=10, total_iterations_limit=None):
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    total_iterations = 0
    best_valid_loss = 100000
    for epoch in range(epochs):
        model.train()

        loss_sum = 0
        num_iterations = 0

        data_iterator = tqdm(train_loader, desc=f'Epoch {epoch+1}')
        if total_iterations_limit is not None:
            data_iterator.total = total_iterations_limit
        for data in data_iterator:
            num_iterations += 1
            total_iterations += 1
            # print(data)
            # print(data[0]['pixel_values'].shape)
            img = torch.squeeze(data[0]['pixel_values'],1)
            text = torch.squeeze(data[1]['input_ids'],1)
            target = torch.squeeze(data[2])
            # print(text.shape)
            img = img.to(device)
            text = text.to(device)
            target = target.to(device)
            optimizer.zero_grad()
            logits = model(img,text)
            print("logits: " + str(torch.sigmoid(logits)))
            print("target: " + str(target))
            loss = F.binary_cross_entropy(torch.sigmoid(logits), target.float())
            # print(loss)
            loss_sum += loss.item()
            avg_loss = loss_sum / num_iterations
            data_iterator.set_postfix(loss=avg_loss)
            loss.backward()
            optimizer.step()

            if total_iterations_limit is not None and total_iterations >= total_iterations_limit:
                return
        if avg_loss < best_valid_loss:
                best_valid_loss = valid_loss
                best_weights = copy.deepcopy(model.state_dict())
                save_checkpoint(checkpoint_directory, epoch, model, optimizer, avg_loss)

        if early_stopper.early_stop(avg_loss):
                print("Early stop at epoch: "+str(epoch) + " with valid loss: "+str(avg_loss))
                break
img_encoder =[vit_encoder,resnet_encoder,swintransformer]

model = CrossModel(img_encoder,encoder_layer,text_embedding, pooling_layer)
train(dataloader_train,model,device ,epochs=100)

Epoch 1:   0%|          | 0/1184 [00:20<?, ?it/s]

image_features_0torch.Size([16, 197, 768])
image_features_1torch.Size([16, 512, 7, 7])
image_features_2torch.Size([16, 49, 768])


RuntimeError: ignored

In [ ]:
# from transformers import AutoImageProcessor, AutoModelForImageClassification
# from PIL import Image
# import requests

# url = "http://images.cocodataset.org/val2017/000000039769.jpg"
# image = Image.open(requests.get(url, stream=True).raw)

# processor = AutoImageProcessor.from_pretrained("microsoft/swin-tiny-patch4-window7-224")
# model = AutoModelForImageClassification.from_pretrained("microsoft/swin-tiny-patch4-window7-224")

# inputs = processor(images=image, return_tensors="pt")
# outputs = model(**inputs)
# logits = outputs.logits
# print(logits.shape)
# # model predicts one of the 1000 ImageNet classes
# # predicted_class_idx = logits.argmax(-1).item()
# # print("Predicted class:", model.config.id2label[predicted_class_idx])

## Predict

In [ ]:
def read_json(parient_dir, name=None):
    # Opening JSON file
    if name == None:
        path = parient_dir
    else:
        path = os.path.join(parient_dir, str(name) + ".json")
    with open(path, 'r') as openfile:

        # Reading from json file
        json_object = json.load(openfile)
    return json_object

json_file = read_json('/content/drive/MyDrive/MMM CONFERENCE/data',name='test')
values = json_file['pairs']
values[0]

In [ ]:
def load_checkpoint(PATH, model, optimizer):
    checkpoint = torch.load(PATH)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']

    return epoch, model, optimizer, loss

In [ ]:
def predict(model,device,test_file_path,image_processor):
  model.eval()
  json_dict = read_json(test_file_path,name='test')
  json_values = json_dict['pairs']
  preds = []
  labels = []
  for item in range(json_values):
    image_url = json_values['image']
    subtask1_label = json_values['json_values']
    labels.appnend()
    image = Image.open(requests.get(image_url, stream=True).raw)
    input = image_processor(image)
    pred = model(input)
    preds.append(pred)
  return preds,labels

